**"Multi-EMAs Strategy for Trend Reversal and Continuation"**

**Hypothesis:**

My strategy is based on the hypothesis that the simultaneous use of multiple exponential moving averages (EMAs) of varying periods along with the closing and opening price of the stocks can effectively identify potential trend reversals and continuations in financial markets. The hypothesis assumes that when these EMAs are in a specific configuration, they can provide insights into the underlying market sentiment and price momentum.

EMAs period used: 20,25,30,35,40,45,50,55

**The strategy seeks two key scenarios:**

Scenario 1:

When all the EMAs are in an uptrend i.e. EMA20>EMA25>EMA30>EMA35>EMA40>EMA45>EMA50>EMA55 enter in the long trade when the close price of the candle is above the EMA20 and open is below EMA55.

Scenario 2:

When all the EMAs are in a downtrend i.e. EMA20<EMA25<EMA30<EMA35<EMA40<EMA45<EMA50<EMA55 enter in the long trade when the close price of the candle is above the EMA55 and open is below EMA20.

The strategy also incorporates volume on the entry candle should be greater than the average of the past 10 candles volume 



**Data source and Backtesting library:**

USED the stock data in the CSV format which is downloaded from the trading view site 
For backtesting the library used is backtrader

In [1]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats
import backtrader.indicators
import datetime
import yfinance as yf
import pandas as pd
import matplotlib as plt

In [47]:
class MAstrategy(bt.Strategy):
    def __init__(self):
        self.ma20 = bt.indicators.ExponentialMovingAverage(self.data.close, period=20)
        self.ma25 = bt.indicators.ExponentialMovingAverage(self.data.close, period=25)
        self.ma30 = bt.indicators.ExponentialMovingAverage(self.data.close, period=30)
        self.ma35 = bt.indicators.ExponentialMovingAverage(self.data.close, period=35)
        self.ma40 = bt.indicators.ExponentialMovingAverage(self.data.close, period=40)
        self.ma45 = bt.indicators.ExponentialMovingAverage(self.data.close, period=45)
        self.ma50 = bt.indicators.ExponentialMovingAverage(self.data.close, period=50)
        self.ma55 = bt.indicators.ExponentialMovingAverage(self.data.close, period=55)
        self.volume_avg = bt.indicators.SimpleMovingAverage(self.data.volume, period=10)
        self.rsi = bt.indicators.RelativeStrengthIndex()
        self.order = None
        self.buy_price = None
        self.stop_price = None
    
    def next(self):
        if self.order:
            return
        if not self.position:
            ma_conditions_asc = [
                self.ma20[0] > self.ma25[0],
                self.ma25[0] > self.ma30[0],
                self.ma30[0] > self.ma35[0],
                self.ma35[0] > self.ma40[0],
                self.ma40[0] > self.ma45[0],
                self.ma45[0] > self.ma50[0],
                self.ma50[0] > self.ma55[0] and self.data.close[0] > self.ma20[0]and self.data.open[0] < self.ma55[0] 
            ]
            
            ma_conditions_desc = [
                self.ma20[0] < self.ma25[0],
                self.ma25[0] < self.ma30[0],
                self.ma30[0] < self.ma35[0],
                self.ma35[0] < self.ma40[0],
                self.ma40[0] < self.ma45[0],
                self.ma45[0] < self.ma50[0],
                self.ma50[0] < self.ma55[0] and 
                self.data.open[0] < self.ma20[0]and self.data.close[0] > self.ma55[0]  # Corrected line
            ]

            if (
                (all(ma_conditions_asc) or all(ma_conditions_desc)) and

                (self.data.volume[0] > self.volume_avg[0])
            ):
                self.log('Buy Create, %.2f' % self.data.close[0])
                self.order = self.buy(size=10)
                self.buy_price = self.data.close[0]
                self.stop_price = self.buy_price * 0.95
                self.log('Stop Loss set at %.2f' % self.stop_price)

        else:
            if self.data.high[0] >= self.buy_price * 1.05:
                self.log('Sell Create, %.2f' % self.data.close[0])
                self.order = self.sell(size=10)
            elif self.data.low[0] <= self.stop_price:
                self.log('Stop Loss triggered, %.2f' % self.data.close[0])
                self.order = self.sell(size=10)

    def log(self, txt):
        dt = self.data.datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    "Executed BUY (Price: %.2f, Value: %.2f, Commission %.2f)" %
                    (order.executed.price, order.executed.value, order.executed.comm))
                self.buy_price = order.executed.price
                self.stop_price = self.buy_price * 0.95
            elif order.issell():
                self.log(
                    "Executed SELL (Price: %.2f, Value: %.2f, Commission %.2f)" %
                    (order.executed.price, order.executed.value, order.executed.comm))

        self.order = None

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(MAstrategy)
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.0001)
    df = pd.read_csv('new50FEDERALBNK1H.csv')
    df['datetime'] = pd.to_datetime(df['datetime'])  # Convert date column to datetime format
    data = bt.feeds.PandasData(dataname=df, datetime='datetime', open='open', high='high', low='low', close='close', volume='volume')
    cerebro.adddata(data)


In [48]:
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='maxdrawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")

In [49]:
def printDrawDownAnalysis(analyzer): # Function to print the Technical Analysis results in a nice format.

    # Get the results we are interested in
    drawdown = round(analyzer.drawdown, 2)
    moneydown = round(analyzer.moneydown, 2)
    length = analyzer.len
    max_dd = round(analyzer.max.drawdown, 2)
    max_md = round(analyzer.max.moneydown, 2)
    max_len = analyzer.max.len
    h1 = ['Drawdown', 'Moneydown', 'Length']
    h2 = ['Max drawdown','Max moneydown', 'Max len']
    r1 = [drawdown, moneydown,length]
    r2 = [max_dd, max_md, max_len]
    # Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    # Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Drawdown Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))


def printTradeAnalysis(analyzer): # Function to print the Technical Analysis results in a nice format.

    # Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = round((total_won / total_closed) * 100)
    # Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed, total_won, total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    # Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    # Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

In [50]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue()) # Print the initial portfolio value
backtest_result = cerebro.run()
backtest_result_1 = backtest_result[0]
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue()) # Print the final portfolio value
print('Sharpe Ratio:', backtest_result_1.analyzers.mysharpe.get_analysis())
printDrawDownAnalysis(backtest_result_1.analyzers.maxdrawdown.get_analysis()) # To get detailed Drawdown and Trade analysis
printTradeAnalysis(backtest_result_1.analyzers.ta.get_analysis())

Starting Portfolio Value: 10000.00
2020-01-20, Buy Create, 93.75
2020-01-20, Stop Loss set at 89.06
2020-01-20, Executed BUY (Price: 93.70, Value: 937.00, Commission 0.09)
2020-02-01, Stop Loss triggered, 90.00
2020-02-01, Executed SELL (Price: 90.00, Value: 937.00, Commission 0.09)
2020-02-27, Buy Create, 87.00
2020-02-27, Stop Loss set at 82.65
2020-02-27, Executed BUY (Price: 86.95, Value: 869.50, Commission 0.09)
2020-03-02, Stop Loss triggered, 82.05
2020-03-02, Executed SELL (Price: 82.15, Value: 869.50, Commission 0.08)
2020-04-22, Buy Create, 44.00
2020-04-22, Stop Loss set at 41.80
2020-04-22, Executed BUY (Price: 44.00, Value: 440.00, Commission 0.04)
2020-04-28, Sell Create, 46.10
2020-04-28, Executed SELL (Price: 46.10, Value: 440.00, Commission 0.05)
2020-07-07, Buy Create, 53.55
2020-07-07, Stop Loss set at 50.87
2020-07-07, Executed BUY (Price: 53.45, Value: 534.50, Commission 0.05)
2020-07-07, Sell Create, 55.90
2020-07-08, Executed SELL (Price: 56.00, Value: 534.50, Co